In [2]:
import pandas as pd
import numpy as np

# Adjust path if needed
df = pd.read_csv("../data/skygeni_sales_data.csv")

# Basic cleaning / types
df["created_date"] = pd.to_datetime(df["created_date"])
df["closed_date"] = pd.to_datetime(df["closed_date"], errors="coerce")

# Target flag
df["won_flag"] = (df["outcome"] == "Won").astype(int)

# Baseline win rate
baseline_wr = df["won_flag"].mean()

print(f"Rows: {len(df)}, baseline win rate: {baseline_wr:.3f}")
df.head()


Rows: 5000, baseline win rate: 0.453


,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome,won_flag
0,D00001,2023-11-24,2023-12-15,rep_22,SaaS,North America,Enterprise,Referral,Qualified,4253,21,Won,1
1,D00002,2023-01-17,2023-01-27,rep_7,SaaS,India,Core,Referral,Closed,3905,10,Won,1
2,D00003,2023-10-29,2023-12-10,rep_5,HealthTech,APAC,Core,Inbound,Proposal,10615,42,Lost,0
3,D00004,2023-07-14,2023-08-02,rep_18,FinTech,India,Core,Partner,Negotiation,4817,19,Won,1
4,D00005,2024-02-29,2024-05-26,rep_2,HealthTech,APAC,Core,Outbound,Qualified,45203,87,Lost,0


In [3]:
# Segment dimensions used by the decision engine
segment_cols_driver = ["industry", "region", "lead_source"]

driver = (
    df.groupby(segment_cols_driver)
      .agg(
          deals=("won_flag", "size"),
          win_rate=("won_flag", "mean"),
          avg_amount=("deal_amount", "mean")
      )
      .reset_index()
)

driver["lift_vs_baseline"] = driver["win_rate"] - baseline_wr
driver["driver_score"] = driver["win_rate"] * driver["avg_amount"]

driver_sorted = driver.sort_values("driver_score", ascending=False)

print("Top 10 driver segments:")
driver_sorted.head(10)


Top 10 driver segments:


,industry,region,lead_source,deals,win_rate,avg_amount,lift_vs_baseline,driver_score
44,FinTech,North America,Inbound,59,0.610169,30933.474576,0.157569,18874.662453
21,EdTech,Europe,Outbound,65,0.569231,31552.738462,0.116631,17960.789586
37,FinTech,Europe,Outbound,44,0.568182,29474.954545,0.115582,16747.133264
75,SaaS,India,Referral,51,0.509804,31112.745098,0.057204,15861.399462
65,SaaS,APAC,Outbound,56,0.464286,34042.625000,0.011686,15805.504464
34,FinTech,APAC,Partner,63,0.523810,29630.746032,0.071210,15520.866969
0,Ecommerce,APAC,Inbound,65,0.492308,31295.600000,0.039708,15407.064615
31,EdTech,North America,Referral,69,0.536232,28040.159420,0.083632,15036.027515
27,EdTech,India,Referral,56,0.535714,28031.410714,0.083114,15016.827168
23,EdTech,Europe,Referral,67,0.447761,33512.955224,-0.004839,15005.800847


In [4]:
# Use (industry, region, lead_source, deal_stage) as segment key for rules
segment_cols_rules = ["industry", "region", "lead_source", "deal_stage"]

seg_rules = (
    df.groupby(segment_cols_rules)
      .agg(
          deals=("won_flag", "size"),
          win_rate=("won_flag", "mean")
      )
      .reset_index()
)

print("Sample of segment rules (win probabilities):")
seg_rules.head()


Sample of segment rules (win probabilities):


,industry,region,lead_source,deal_stage,deals,win_rate
0,Ecommerce,APAC,Inbound,Closed,13,0.461538
1,Ecommerce,APAC,Inbound,Demo,13,0.538462
2,Ecommerce,APAC,Inbound,Negotiation,9,0.666667
3,Ecommerce,APAC,Inbound,Proposal,16,0.687500
4,Ecommerce,APAC,Inbound,Qualified,14,0.142857


In [5]:
# Merge segment win_rate back to deals
df_scored = df.merge(
    seg_rules,
    on=segment_cols_rules,
    how="left",
    suffixes=("", "_seg")
)

# Fallback to baseline if no segment history
df_scored["seg_win_prob"] = df_scored["win_rate"].fillna(baseline_wr)

def risk_band(p, baseline=baseline_wr, delta=0.05):
    if p >= baseline + delta:
        return "low_risk_high_fit"
    elif p <= baseline - delta:
        return "high_risk_low_fit"
    else:
        return "medium_risk"

df_scored["risk_band"] = df_scored["seg_win_prob"].apply(risk_band)

df_scored[[
    "deal_id", "industry", "region", "lead_source", "deal_stage",
    "seg_win_prob", "risk_band"
]].head()


,deal_id,industry,region,lead_source,deal_stage,seg_win_prob,risk_band
0,D00001,SaaS,North America,Referral,Qualified,0.500000,medium_risk
1,D00002,SaaS,India,Referral,Closed,0.888889,low_risk_high_fit
2,D00003,HealthTech,APAC,Inbound,Proposal,0.500000,medium_risk
3,D00004,FinTech,India,Partner,Negotiation,0.416667,medium_risk
4,D00005,HealthTech,APAC,Outbound,Qualified,0.416667,medium_risk


In [6]:
# Alert 1: Win-rate drop in key segments over time
df["closed_date"] = pd.to_datetime(df["closed_date"], errors="coerce")

recent_days = 30
history_days = 90
min_deals = 30
drop_threshold = 0.08  # 8 percentage points

max_date = df["closed_date"].max()
recent_start = max_date - pd.Timedelta(days=recent_days)
history_start = max_date - pd.Timedelta(days=history_days)

recent = df[
    (df["closed_date"] >= recent_start) &
    (df["closed_date"] <= max_date)
].copy()

history = df[
    (df["closed_date"] >= history_start) &
    (df["closed_date"] < recent_start)
].copy()

recent_seg = (
    recent.groupby(segment_cols_driver)["won_flag"]
    .agg(["size", "mean"])
    .rename(columns={"size": "recent_deals", "mean": "recent_wr"})
    .reset_index()
)

hist_seg = (
    history.groupby(segment_cols_driver)["won_flag"]
    .agg(["size", "mean"])
    .rename(columns={"size": "hist_deals", "mean": "hist_wr"})
    .reset_index()
)

wr_change = (
    recent_seg.merge(hist_seg, on=segment_cols_driver, how="inner")
    .query("recent_deals >= @min_deals and hist_deals >= @min_deals")
)

wr_change["drop"] = wr_change["hist_wr"] - wr_change["recent_wr"]

alerts_win_drop = wr_change[wr_change["drop"] >= drop_threshold].copy()

print("Win-rate drop alerts:")
if alerts_win_drop.empty:
    print("None triggered with current thresholds.")
else:
    for _, row in alerts_win_drop.iterrows():
        print(
            f"[ALERT: WIN RATE DROP] {row['industry']} – {row['region']} via {row['lead_source']}: "
            f"{row['hist_wr']:.1%} → {row['recent_wr']:.1%} "
            f"on {int(row['recent_deals'])} recent deals (drop {row['drop']:.1%})."
        )


Win-rate drop alerts:
None triggered with current thresholds.


In [7]:
# Alert 2: Emerging high-value driver segments
min_deals_driver = 40
lift_threshold = 0.05  # 5 points above baseline
top_n = 5

candidate = driver.query("deals >= @min_deals_driver and lift_vs_baseline >= @lift_threshold")

top_emerging = candidate.sort_values("driver_score", ascending=False).head(top_n)

print("\nEmerging high-value driver alerts:")
if top_emerging.empty:
    print("None triggered with current thresholds.")
else:
    for _, row in top_emerging.iterrows():
        print(
            f"[ALERT: EMERGING DRIVER] {row['industry']} – {row['region']} via {row['lead_source']}: "
            f"win rate {row['win_rate']:.1%} on {int(row['deals'])} deals, "
            f"avg deal {row['avg_amount']:.0f} (lift vs baseline {row['lift_vs_baseline']:.1%})."
        )



Emerging high-value driver alerts:
[ALERT: EMERGING DRIVER] FinTech – North America via Inbound: win rate 61.0% on 59 deals, avg deal 30933 (lift vs baseline 15.8%).
[ALERT: EMERGING DRIVER] EdTech – Europe via Outbound: win rate 56.9% on 65 deals, avg deal 31553 (lift vs baseline 11.7%).
[ALERT: EMERGING DRIVER] FinTech – Europe via Outbound: win rate 56.8% on 44 deals, avg deal 29475 (lift vs baseline 11.6%).
[ALERT: EMERGING DRIVER] SaaS – India via Referral: win rate 51.0% on 51 deals, avg deal 31113 (lift vs baseline 5.7%).
[ALERT: EMERGING DRIVER] FinTech – APAC via Partner: win rate 52.4% on 63 deals, avg deal 29631 (lift vs baseline 7.1%).


In [8]:
# Alert 3: Pipeline skewed toward low-fit segments
# If you had a separate "open pipeline" view, filter it; here we treat all non-closed outcome as open.
# If your data marks only Won/Lost, skip the filter and just demonstrate on df_scored.

if "outcome" in df_scored.columns and df_scored["outcome"].notna().all():
    # demo-only: assume df_scored is current pipeline
    open_pipeline = df_scored.copy()
else:
    open_pipeline = df_scored.copy()

total_value = open_pipeline["deal_amount"].sum()
low_fit_value = open_pipeline.loc[
    open_pipeline["risk_band"] == "high_risk_low_fit", "deal_amount"
].sum()

low_fit_share = low_fit_value / total_value if total_value > 0 else 0.0
threshold_low_fit_share = 0.4  # 40%

print("\nPipeline composition alert:")
if low_fit_share >= threshold_low_fit_share:
    print(
        f"[ALERT: PIPELINE LOW-FIT SKEW] "
        f"{low_fit_share:.1%} of pipeline value is in low-fit segments "
        f"(threshold {threshold_low_fit_share:.0%})."
    )
else:
    print(
        f"No alert: low-fit segments account for {low_fit_share:.1%} of pipeline value "
        f"(threshold {threshold_low_fit_share:.0%})."
    )



Pipeline composition alert:
No alert: low-fit segments account for 36.0% of pipeline value (threshold 40%).


In [9]:
# Alert 4: Stage stalling by segment
# Use closed deals to learn typical sales_cycle_days by (industry, region, deal_stage)
closed = df[df["outcome"].notna()].copy()

stall_segment_cols = ["industry", "region", "deal_stage"]

seg_cycle = (
    closed.groupby(stall_segment_cols)["sales_cycle_days"]
    .agg(["median"])
    .rename(columns={"median": "median_cycle"})
    .reset_index()
)

# Join median cycle back to (demo) open pipeline
open_pipeline = open_pipeline.merge(seg_cycle, on=stall_segment_cols, how="left")

margin_days = 15

open_pipeline["is_stalled"] = (
    (open_pipeline["sales_cycle_days"] > open_pipeline["median_cycle"] + margin_days)
    & open_pipeline["median_cycle"].notna()
)

stall_summary = (
    open_pipeline[open_pipeline["is_stalled"]]
    .groupby(stall_segment_cols)
    .agg(
        stalled_deals=("deal_id", "size"),
        stalled_value=("deal_amount", "sum")
    )
    .reset_index()
)

min_stalled_deals = 10

print("\nStage stalling alerts:")
stall_filtered = stall_summary.query("stalled_deals >= @min_stalled_deals")
if stall_filtered.empty:
    print("None triggered with current thresholds.")
else:
    for _, row in stall_filtered.iterrows():
        print(
            f"[ALERT: STALLING] {row['industry']} – {row['region']} at stage {row['deal_stage']}: "
            f"{int(row['stalled_deals'])} stalled deals worth {row['stalled_value']:.0f}. "
            f"Deals are beyond median cycle + {margin_days} days."
        )



Stage stalling alerts:
[ALERT: STALLING] Ecommerce – APAC at stage Closed: 18 stalled deals worth 476443. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – APAC at stage Demo: 19 stalled deals worth 589242. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – APAC at stage Negotiation: 18 stalled deals worth 514246. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – APAC at stage Proposal: 18 stalled deals worth 376871. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – APAC at stage Qualified: 17 stalled deals worth 433969. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – Europe at stage Closed: 14 stalled deals worth 331007. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – Europe at stage Demo: 19 stalled deals worth 625217. Deals are beyond median cycle + 15 days.
[ALERT: STALLING] Ecommerce – Europe at stage Negotiation: 14 stalled deals worth 488512. Deals ar